## Clustering Anomaly Detection

In [1]:
# imports
# from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

from env import host, user, password
import acquire
import prepare

#### Clustering - DBSCAN
- Use DBSCAN to detect anomalies in other products from the customers dataset. The database name is grocery_db.
- Use DBSCAN to detect anomalies in requests from the curriculum access logs in the curriculum_logs database.
- Use DBSCAN to detect anomalies in number of bedrooms and finished square feet of property for the filtered dataset you used in the clustering project (single unit properties with a logerror).

# Curriculum Logs

### Bring in the cohorts table...

In [2]:
cohorts_df = acquire.get_cohort_data()

In [3]:
# let's take a peak..
cohorts_df.head()

,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,3,Carlsbad,#carlsbad,2014-09-04,2014-11-05,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,4,Denali,#denali,2014-10-20,2015-01-18,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,5,Everglades,#everglades,2014-11-18,2015-02-24,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1


In [4]:
cohorts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          53 non-null     int64  
 1   name        53 non-null     object 
 2   slack       53 non-null     object 
 3   start_date  53 non-null     object 
 4   end_date    53 non-null     object 
 5   created_at  53 non-null     object 
 6   updated_at  53 non-null     object 
 7   deleted_at  0 non-null      float64
 8   program_id  53 non-null     int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 4.1+ KB


#### Cohorts Table Key Findings & Takeaways:
- 9 columns by 53 rows
- need to convert all date columns to datetime
- looks like I can probably get rid of 'deleted_at' since all are null
- do I have any reason to keep 'slack'?
- otherwise, each variable has no null-values

In [5]:
cohorts_df = prepare.initial_cohorts_prep(cohorts_df)

In [6]:
cohorts_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 0 to 52
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          53 non-null     int64         
 1   name        53 non-null     object        
 2   start_date  53 non-null     datetime64[ns]
 3   end_date    53 non-null     datetime64[ns]
 4   created_at  53 non-null     datetime64[ns]
 5   updated_at  53 non-null     datetime64[ns]
 6   program_id  53 non-null     int64         
dtypes: datetime64[ns](4), int64(2), object(1)
memory usage: 3.3+ KB


### Bring in the curriculum logs table...

In [7]:
curr_df = acquire.get_curriculum_logs()

In [8]:
# let's take a peak..
curr_df.head()

,date,endpoint,user_id,cohort_id,source_ip
0,2018-01-26,/,1,8.0,97.105.19.61
1,2018-01-26,java-ii,1,8.0,97.105.19.61
2,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61


In [9]:
curr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018810 entries, 0 to 1018809
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   date       1018810 non-null  object 
 1   endpoint   1018809 non-null  object 
 2   user_id    1018810 non-null  int64  
 3   cohort_id  965313 non-null   float64
 4   source_ip  1018810 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 38.9+ MB


#### Curriculum Logs Table Key Findings & Takeaways:
- 5 columns by 1_018_810 rows
- need to convert date column to datetime
- 'endpoint' has one null-value... what to do here?
- 'cohort_id' can be converted to int
- 'cohort_id' has 53_497 null values...
    - are these instructors?
    - hackers?
    - other?

In [10]:
curr_df = prepare.initial_curr_prep(curr_df)

In [11]:
curr_df.head()

,date,endpoint,user_id,cohort_id,source_ip
date,,,,,
2018-01-26,2018-01-26,/,1,8.0,97.105.19.61
2018-01-26,2018-01-26,java-ii,1,8.0,97.105.19.61
2018-01-26,2018-01-26,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26,2018-01-26,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26,2018-01-26,javascript-i/conditionals,2,22.0,97.105.19.61


In [13]:
curr_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1018810 entries, 2018-01-26 to 2021-07-15
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype         
---  ------     --------------    -----         
 0   date       1018810 non-null  datetime64[ns]
 1   endpoint   1018809 non-null  object        
 2   user_id    1018810 non-null  int64         
 3   cohort_id  965313 non-null   float64       
 4   source_ip  1018810 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 46.6+ MB


## Merge the Tables

In [ ]:
df = prepare.curr_cohort_join(curr_df, cohorts_df)